# Run 'nibetaseries' package on all subjects

In [1]:
import os  # interact with the filesystem
from subprocess import Popen, PIPE, STDOUT  # enable calling commandline
import matplotlib.pyplot as plt  # manipulate figures
import seaborn as sns  # display results
import pandas as pd   # manipulate tabular 
import warnings
import time
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.filterwarnings("ignore", category=ResourceWarning) 

## I. Data directory set up

## II. TSV files set up   
Origianl Betaseries Events file: `sub-001_run01_task-pst_events.tsv`  
BIDS events file: `sub-001_task-prob_run-1_events.tsv`

#### Go to setup_tsvs.py to see example code 

In [ ]:
data_dir = '/projects/niblab/bids_projects/Experiments/Bevel/BIDS' #RENCI PATH
#data_dir = '/Users/nikkibytes/Documents/niblunc/testing_beta' #LOCAL PATH
print('Our working directory: {}'.format(data_dir))

In [ ]:
atlas_txt = os.path.join(data_dir, 'derivatives', 'parcellations','WashU_300rois_order_new.txt')
atlas_df = pd.read_csv(atlas_txt, sep="\t")
atlas_df.head()

atlas_df.drop(['network', 'x_coord', 'y_coord', 'z_coord'], axis='columns', inplace=True)
atlas_df.tail()

atlas_df = atlas_df.append({'index' : '300', 'regions' : 'pHippocampusL'}, ignore_index=True)
atlas_df.tail()
atlas_tsv = atlas_txt.replace(".txt", ".tsv")
atlas_df.to_csv(atlas_tsv, sep="\t", index=False)

In [ ]:
out_dir = os.path.join(data_dir, "derivatives")
work_dir = os.path.join(out_dir, "work_dir")
atlas_mni_file = os.path.join(data_dir,
                              "derivatives",
                              "parcellations",
                              "WashU_300rois_MNI152_3mm_origOrder.nii")
print('Our output directory: {}'.format(out_dir))
print('Our working output directory: {}'.format(work_dir))

In [ ]:
# Run parallel
# 
cmd = """\
nibs \
{bids_dir} \
fmriprep \
{out_dir} \
participant \
--participant_label 001 \
-sm 6 \
--run_label 1 \
-c CSF FramewiseDisplacement X Y Z RotX RotY RotZ \
-sp MNI152NLin2009cAsym \
-w {work_dir} \
-a {atlas_mni_file} \
-l {atlas_tsv} \
--nthreads 16
""".format(atlas_mni_file=atlas_mni_file,
           atlas_tsv=atlas_tsv,
           bids_dir=os.path.join(data_dir),
           out_dir=out_dir,
           work_dir=work_dir)





# Since we cannot run bash commands inside this tutorial
# we are printing the actual bash command so you can see it
# in the output
print("The Example Command:\n", cmd)



In [ ]:
# call nibs

p = Popen(cmd, shell=True, stdout=PIPE, stderr=STDOUT)

while True:
    line = p.stdout.readline()
    if not line:
        break
    print(line)
    
    

In [ ]:
corr_mat_path = os.path.join(out_dir, "NiBetaSeries", "nibetaseries", "sub-001", "func")
trial_types = ['punish','reward']
filename_template = "sub-001_task-prob_run-1_bold_space-MNI152NLin2009cAsym_preproc_trialtype-{trial_type}_matrix.tsv"
pd_dict = {}

for trial_type in trial_types:
    file_path = os.path.join(corr_mat_path, filename_template.format(trial_type=trial_type))
    print(file_path)
    pd_dict[trial_type] = pd.read_csv(file_path, sep='\t', na_values="n/a", index_col=0)
# display example matrix
    print(pd_dict[trial_type].head())